In [1]:
#import libraries
import numpy as np
import cv2
import Image_Processing_Classes as ip
from matplotlib import pyplot as plt
import mediapipe as mp
import math
import os
import imutils
from IPython.display import Image

In [ ]:
#reading in and pre-processing image to be used with mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_styles = mp.solutions.drawing_styles

#help(mp_pose.Pose)

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480
def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2.imshow('pre-processed image', img)
  cv2.waitKey(0)

# Read images with OpenCV.
names = ['messi_original']
images = {name: cv2.imread('Images/messi2.png') for name in names}

# Preview the image.
plt.imshow(cv2.cvtColor(images['messi_original'], cv2.COLOR_BGR2RGB))

 


In [4]:
def initialise_yolo():
    """
    Function to initialise the weights, architecture, and labels for the yolo model

    :return: yolo model, yolo model output layer names, model labels (ball, person, etc.)
    """
    yolo_dir = os.path.abspath("./yolo")

    # load the labels, weights, and config for the yolo model
    weights_path = os.path.sep.join([yolo_dir, "yolov3.weights"])
    config_path = os.path.sep.join([yolo_dir, "yolov3.cfg"])

    # load the labels (as list), and model
    yolo_model = cv2.dnn.readNetFromDarknet(config_path, weights_path)

    # get the output layers
    layer_names = yolo_model.getLayerNames()
    layer_names = [layer_names[i - 1] for i in yolo_model.getUnconnectedOutLayers()]

    return yolo_model, layer_names

In [5]:
#method for detecting ball on image
def find_ball(frame):
    #frame = imutils.resize(frame, width=500)
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    yolo_model, ln = initialise_yolo()
    yolo_model.setInput(blob)
    layerOutputs = yolo_model.forward(ln)
    boxes = []
    confidences = []
    classIDs = []
    confidence_threshold = 0.3
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # get the class id and confidence for the object
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > confidence_threshold and classID == 32:
                # get the bounding box for the object
                (H, W) = frame.shape[:2]
                box = detection[0:4] * np.array([W, H, W, H])
                (x, y, width, height) = box.astype("int")
                # update our list of bounding box coordinates, confidences, and class IDs
                boxes.append([x, y, width, height])
                confidences.append(float(confidence))
                classIDs.append(classID)
    (x, y, w, h) = boxes[np.argmax(confidences)]
    return x, y, w, h
    
x, y, w, h = find_ball(frame)
plt.imshow(cv2.circle(frame.copy(), center = (x, y), radius=int(w/2), color=(0,255,0), thickness=3))    
    
       


NameError: name 'frame' is not defined

In [ ]:
def get_body_vectors(image, mp_pose):
    with mp_pose.Pose(
        static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:
        # Convert the BGR image to RGB and process it with MediaPipe Pose.
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Draw pose landmarks.
        #print(f'Pose landmarks of {name}:')
        annotated_image = image.copy()
        mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style())
        #annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        plt.imshow(annotated_image)    

In [ ]:
# apply non-maxima suppression to refine the balls bounding box
'''
vs = cv2.VideoCapture('clip.mp4')

while True:
    # grab the next frame in the video stream
    (grabbed, frame) = vs.read()

    # check to see if we have reached the end of the stream
    if frame is None:
        break

    # resize the frame (so we can process it faster)
    frame = imutils.resize(frame, width=500)
    (H, W) = frame.shape[:2]




frame = imutils.resize(annotated_image, width=500)
blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
yolo_model, ln = initialise_yolo()
yolo_model.setInput(blob)
layerOutputs = yolo_model.forward(ln)
boxes = []
confidences = []
classIDs = []
confidence_threshold = 0.3
suppression_threshold = 0.3


for output in layerOutputs:
    # loop over each of the detections
    for detection in output:
        # get the class id and confidence for the object
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > confidence_threshold and classID == 32:
            # get the bounding box for the object
            (H, W) = frame.shape[:2]
            box = detection[0:4] * np.array([W, H, W, H])
            (x, y, width, height) = box.astype("int")
            # update our list of bounding box coordinates, confidences, and class IDs
            boxes.append([x, y, width, height])
            confidences.append(float(confidence))
            classIDs.append(classID)


(x, y, w, h) = boxes[np.argmax(confidences)]
frame = frame.copy()
cv2.circle(frame, center=(x, y), radius=int(w/2), color=(0, 255, 0), thickness=3)
cv2.imshow('out', frame)
cv2.waitKey(0)
if cv2.imwrite('Images/messi2_annotated.png', frame):
    print('success!')




idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, suppression_threshold)

# ensure at least one detection exists
if len(idxs) > 0:
    # loop over the indexes we are keeping
    for i in idxs.flatten():
        # we only care about the ball
        if labels[classIDs[i]] == "sports ball":
            print('found a ball')
            # let the system know we found the ball
            ball_found_initially = True
            ball_found_in_frame = True
            object_lost_count = 0
            prediction_count = 0
            
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            cv2.circle(img=images['messi_original'], center=(y, x), radius=int(h/4), color=(255, 0, 0))
            
            # calculate the centre of the bounding box to draw the trace on the frame
            trace_location = (int(x + (w / 2)), int(y + (h / 2)))
            cv2.imshow('out', images['messi_original'])
            cv2.waitKey(0)
            #print(boxes[i])
    '''